In [ ]:
!pip install ultralytics

In [ ]:
# connect google drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# set the working directory
import os

os.chdir(r'/content/gdrive/MyDrive/YOLO Moving Object/')
os.environ["TORCH_CUNN_SDPA_ENABLED"] = "1"
HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolo11n.pt")

# Define input and output video paths
SOURCE_VIDEO_PATH = "vehicles.mp4"  # Your input video
OUTPUT_VIDEO_PATH = "vehicles-detection.mp4"  # Save the output video

# Open video file
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, int(cap.get(5)),
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv11 on the current frame
    results = model(frame)[0]

    # Draw bounding boxes
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        label = results.names[int(box.cls[0])]  # Class label
        confidence = box.conf[0]  # Confidence score

        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the processed frame
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
